### Creating and Loading Date table

In [0]:
def load_date_table(start_date = '2023-01-01', end_date = '2023-12-31'):
    #importing libraries
    from pyspark.sql import functions as fn
    from pyspark.sql.types import IntegerType
    from pyspark.pandas import date_range
    
    # creating the date range to load into the date table
    date_df = spark.createDataFrame(date_range(start_date, end_date).to_numpy(),['Date'])
    
    # creating all the necessary attributes required for date_table
    date_df = date_df.withColumn("date_key", fn.date_format('Date', 'yyyyMMdd').cast(IntegerType()))
    date_df = date_df.withColumn("full_date", fn.date_format('Date', 'yyyy-MM-dd').cast('DATE'))
    date_df = date_df.withColumn("month_name", fn.date_format('Date', 'MMMM'))
    date_df = date_df.withColumn("month_num_of_year", fn.month('Date'))
    date_df = date_df.withColumn("day_num_of_week", fn.date_format('Date', 'EEEE'))
    date_df = date_df.withColumn("day_name_of_week", fn.dayofweek('Date'))
    date_df = date_df.withColumn("day_num_of_month", fn.dayofmonth('Date'))
    date_df = date_df.withColumn("day_num_of_year", fn.dayofyear('Date'))
    date_df = date_df.withColumn("week_num_of_year", fn.weekofyear('Date'))
    date_df = date_df.withColumn("quarter_name", fn.concat(fn.lit("Q"), fn.quarter('Date')))
    date_df = date_df.withColumn("calendar_quarter", fn.quarter('Date'))
    date_df = date_df.withColumn("calendar_year", fn.year('Date'))
    date_df = date_df.drop('Date')
    
    # loading the created dataframe into dim_date_table
    date_df.write.format('delta').mode('overwrite').saveAsTable('dim_date_table')

### Creating and loading time table

In [0]:
def load_time_table():
    # loading required libraries
    from pyspark.sql.functions import col
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType
    
    # creating the time_table schema
    schema = StructType([
    StructField("timeID", IntegerType(), nullable=False),
    StructField("startTime", StringType(), nullable=False),
    StructField("endTime", StringType(), nullable=False)
    ])
    
    # creating the data to load into the time table
    time_ranges = []
    for hour in range(24):
        start_time = f"{hour:02d}:00"
        end_time = f"{hour:02d}:59"
        time_ranges.append((hour, start_time, end_time))
        
    # loading the data into dim_time_table
    time_df = spark.createDataFrame(time_ranges, schema)
    time_df.write.format('DELTA').mode('overwrite').saveAsTable('dim_time_table')